In [ ]:
# =============================================================================
# Project: Trend-Context Fusion Network (TCFN) for PV Power Forecasting
# File: ablation_study_variants.ipynb
# Description: This script performs an ablation study to analyze the impact of
#              removing specific modules (Conv1D, Attention, LSTM, Cyclical Encoding)
#              from the architecture.
# Environment: Google Colab / TensorFlow 2.x
# =============================================================================

# ---------------------------------------------------------
# 0. Setup & Configuration
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import tensorflow as tf
import io
import matplotlib.pyplot as plt
import os
import random
from google.colab import files
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Dropout, LayerNormalization,
                                     MultiHeadAttention, Add, Concatenate, LSTM,
                                     Conv1D, BatchNormalization, GlobalAveragePooling1D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration for Reproducibility
SEED = 42
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

# Hyperparameters & Constants
SEQ_LENGTH = 24
H_PARAMS = {
    'lstm_units': 128,
    'dropout': 0.5,
    'lr': 0.0005,
    'n_heads': 4   # For Attention Mechanism
}

In [ ]:
# ---------------------------------------------------------
# 1. Data Loading & Preprocessing
# ---------------------------------------------------------
print("\n--- [Step 1] Data Upload ---")
print("Please upload your dataset file.")
print("Recommended files: 'Dangjin_Landfill_PV_Dataset.csv' or 'Gwangyang_Port_Phase2_PV.csv'")

uploaded = files.upload()
filename = next(iter(uploaded))
print(f" >> File '{filename}' uploaded successfully.")

df = pd.read_csv(io.BytesIO(uploaded[filename]))

def add_cyclical_features(df):
    """
    Adds cyclical time encoding (Sine/Cosine) for Day of Year and Hour of Day.
    This helps the model capture seasonal and diurnal patterns explicitly.
    """
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['DaysInYear'] = df['Date'].dt.is_leap_year.apply(lambda x: 366 if x else 365)

    # Yearly Seasonality
    df['Day_sin'] = np.sin(2 * np.pi * df['DayOfYear'] / df['DaysInYear'])
    df['Day_cos'] = np.cos(2 * np.pi * df['DayOfYear'] / df['DaysInYear'])

    # Daily Seasonality
    df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24.0)
    df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24.0)
    return df

# Apply Feature Engineering
df = add_cyclical_features(df)

# Feature Definition
weather_cols = ['AverageTemp', 'LowTemp', 'HighTemp', 'RainFall', 'SteamPress',
                'DewPoint', 'Sunshine', 'Insolation', 'Cloudiness', 'GroundTemp',
                'Temp', 'Wind', 'Press', 'Humi']
time_cols = ['Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos']
full_features = weather_cols + time_cols
target_col = 'Solar_Power'

# Data Partitioning (Chronological Split)
# Train: 2015-2017 | Val: 2018 | Test: 2019
train_df = df[df['Year'].isin([2015, 2016, 2017])]
val_df = df[df['Year'] == 2018]
test_df = df[df['Year'] == 2019]

# Standardization
scaler = StandardScaler()
X_train_raw = scaler.fit_transform(train_df[full_features].values)
X_val_raw = scaler.transform(val_df[full_features].values)
X_test_raw = scaler.transform(test_df[full_features].values)

y_train = train_df[target_col].values
y_val = val_df[target_col].values
y_test = test_df[target_col].values

def create_sequences(data, target, seq_length):
    """Generates time-series sequences for LSTM input."""
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(target[i + seq_length])
    return np.array(xs), np.array(ys)

In [ ]:
# ---------------------------------------------------------
# 2. Model Architecture (Flexible Builder)
# ---------------------------------------------------------
def build_ablation_model(input_shape, model_type='TCFN'):
    """
    Constructs the model architecture based on the specified variant.

    Args:
        input_shape: Shape of the input data (Sequence Length, Features).
        model_type: Variant name ('TCFN', 'No_Conv', 'No_Attn', 'No_LSTM').
    """
    inputs = Input(shape=input_shape)

    # --- Module 1: Local Trend Extraction (Conv1D) ---
    if model_type != 'No_Conv':
        trend = Conv1D(H_PARAMS['lstm_units'], kernel_size=3, padding='same', activation='swish')(inputs)
        trend = BatchNormalization()(trend)
        trend = Dropout(H_PARAMS['dropout'])(trend)
    else:
        trend = None

    # --- Module 2: Global Context Mining (Multi-Head Attention) ---
    if model_type != 'No_Attn':
        context = MultiHeadAttention(num_heads=H_PARAMS['n_heads'], key_dim=16)(inputs, inputs)
        context = Add()([inputs, context]) # Residual Connection
        context = LayerNormalization()(context)
        ffn_c = Dense(inputs.shape[-1], activation='swish')(context)
        context = Add()([context, ffn_c])
        context = LayerNormalization()(context)
    else:
        context = inputs

    # --- Feature Fusion Strategy ---
    if model_type == 'No_Conv':
        combined = context
    elif model_type == 'No_Attn':
        combined = trend
    else:
        # Concatenate both pathways (for 'No_LSTM' or 'No_Cyclic' variants)
        combined = Concatenate()([trend, context])

    # --- Module 3: Sequential Modeling (LSTM) ---
    if model_type != 'No_LSTM':
        x = LSTM(H_PARAMS['lstm_units'])(combined)
    else:
        # Ablation: Replace LSTM with Global Average Pooling
        x = GlobalAveragePooling1D()(combined)
        x = Dense(H_PARAMS['lstm_units'], activation='swish')(x)

    x = Dropout(H_PARAMS['dropout'])(x)

    # --- Regression Head ---
    x = Dense(H_PARAMS['lstm_units'], activation='swish')(x)
    outputs = Dense(1, activation='relu')(x) # ReLU for non-negative power output

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=H_PARAMS['lr']), loss='mse', metrics=['mae'])
    return model

In [ ]:
# ---------------------------------------------------------
# 3. Experiment Execution (Ablation Variants Only)
# ---------------------------------------------------------
print("\n--- [Step 2] Conducting Ablation Study (Variants Only) ---")

# Define Ablation Scenarios (Excluding the Full Proposed Model)
experiments = [
    {'name': 'w/o Conv1D',        'type': 'No_Conv', 'use_cyclic': True},
    {'name': 'w/o Attention',     'type': 'No_Attn', 'use_cyclic': True},
    {'name': 'w/o LSTM',          'type': 'No_LSTM', 'use_cyclic': True},
    {'name': 'w/o Cyclic Enc.',   'type': 'TCFN',    'use_cyclic': False}
]

results = []

for exp in experiments:
    print(f"\n>> Running Experiment: {exp['name']} ...")

    # Select Features (Handle 'w/o Cyclic Enc.' case)
    if exp['use_cyclic']:
        features_idx = list(range(len(full_features)))
    else:
        # Exclude the last 4 cyclical time features
        features_idx = list(range(len(weather_cols)))

    # Prepare Data for this specific experiment
    X_train_exp, y_train_exp = create_sequences(X_train_raw[:, features_idx], y_train, SEQ_LENGTH)
    X_val_exp, y_val_exp = create_sequences(X_val_raw[:, features_idx], y_val, SEQ_LENGTH)
    X_test_exp, y_test_exp = create_sequences(X_test_raw[:, features_idx], y_test, SEQ_LENGTH)

    # Build Model
    model = build_ablation_model(input_shape=(SEQ_LENGTH, len(features_idx)), model_type=exp['type'])

    # Training with Early Stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train_exp, y_train_exp,
        validation_data=(X_val_exp, y_val_exp),
        epochs=30,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0 # Suppress epoch logs for cleaner output
    )

    # Evaluation
    y_pred = model.predict(X_test_exp, verbose=0)
    rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred))
    mae = mean_absolute_error(y_test_exp, y_pred)
    r2 = r2_score(y_test_exp, y_pred)

    print(f"   -> [Result] RMSE: {rmse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f}")

    results.append({
        'Model Variant': exp['name'],
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    })

In [ ]:
# ---------------------------------------------------------
# 4. Final Reporting & Visualization
# ---------------------------------------------------------
print("\n--- [Step 3] Final Experimental Report ---")

# Create DataFrame
results_df = pd.DataFrame(results)
print("\n[Table: Ablation Variants Results]")
print(results_df)

# Visualization
plt.figure(figsize=(10, 6))
# Colors: All gray since we are comparing variants
bars = plt.bar(results_df['Model Variant'], results_df['RMSE'], color='gray', alpha=0.85)

plt.title('Ablation Study: Performance of Reduced Variants (RMSE)', fontsize=14, fontweight='bold')
plt.ylabel('RMSE (kW) - Lower is Better', fontsize=12)
plt.xlabel('Ablation Variants', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Add value labels on top of bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1.0, f"{yval:.2f}",
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

print("\n[Summary Analysis]")
print("1. All variants represent partial architectures of the TCFN framework.")
print("2. The RMSE values indicate the error magnitude introduced by removing each specific component.")
print("3. High error rates in specific variants (e.g., 'w/o LSTM') highlight critical dependencies for the dataset.")